In [5]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

from datasets_local.metadata import create_folder_csv
import pandas as pd
import numpy as np

In [24]:
folder_name = 'breast40k_RGB'
folder_path = repo_path / 'data/images' / folder_name
new_folder_path = repo_path / 'experiments/inference/data/original' / f'{folder_name}-size'
new_folder_path.mkdir(parents=False, exist_ok=True)

# get json line file inside folder path
json_file = [f for f in folder_path.glob('*.jsonl')][0]
# read json line file
df = pd.read_json(json_file, lines=True)

# HP
view = 'MLO'
size = 'small'
num_samples = 4

# criteria prompt
criteria_prompt = f'mammogram, {view} view, {size} size'
# make criteria prompt qualified for file name
criteria_prompt_save = criteria_prompt.replace(' ', '_')
criteria_prompt_save = criteria_prompt_save.replace(',', '')

# filter only if critaria prompt is included in the df prompt
df_filtered = df[df['prompt'].str.contains(criteria_prompt)]
# get random samples
df_filtered = df_filtered.sample(num_samples)
df_filtered.reset_index(inplace=True)
for i, row in df_filtered.iterrows():
    image_path = folder_path / row['file_name']
    # copy file to new folder
    new_image_path = new_folder_path / f'{criteria_prompt_save}_{i}.png'
    os.system(f'cp {image_path} {new_image_path}')